In [ ]:
#!/usr/bin/env python3
"""
Adapted from “Cerebral chemoarchitecture shares organizational traits with brain structure and function” [Hänisch et al. eLife 2023] 

by Carlos Estevez-Fraga
"""

In [ ]:
import enigmatoolbox
from enigmatoolbox import datasets
from enigmatoolbox.utils.parcellation import parcel_to_surface
import nilearn
from brainspace.null_models import SpinPermutations
import matplotlib.pyplot as plt
import plotly.express as px
from mn_funcs import spin, schaefer_to_surf
from plotly.graph_objs import *
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
import re
import os
from nilearn import surface

In [ ]:
parcels = 100
input_path = '/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/data/'
res_p = '/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/results/F3/'

#rc_g1 = np.load(input_path + 'rc_g1_{}.npy')
#rc_g2 = np.load(input_path + 'rc_g2_{}.npy')
#rc_g3 = np.load(input_path + 'rc_g3_{}.npy')

rc_g1 = np.load(input_path + 'rc_g1_100.npy')
rc_g2 = np.load(input_path + 'rc_g2_100.npy')
rc_g3 = np.load(input_path + 'rc_g3_100.npy')



In [ ]:
#NODE
disorders = ['node_track', 'node_trackon', 'node_yas']
d = {}


# Read the CSV file
data=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/R/mind/track/results/node/cohens_d_right_transp.csv', header = None)
# Remove the first row
data = data.iloc[1:]
# Remove the first two columns
data = data.iloc[:, 1:]
# Reset the index
data = data.reset_index(drop=True)
data=data.values.flatten()
ct_surf = parcel_to_surface(data, target_lab='aparc_fsa5')
d['node_track']=ct_surf 



# Read the CSV file
data=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/R/mind/trackon/results/node/cohens_d_right_transp.csv', header = None)
# Remove the first row
data = data.iloc[1:]
# Remove the first two columns
data = data.iloc[:, 1:]
# Reset the index
data = data.reset_index(drop=True)
data=data.values.flatten()
ct_surf = parcel_to_surface(data, target_lab='aparc_fsa5')
d['node_trackon']=ct_surf 

# Read the CSV file
data=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/R/mind/yas/results/node/cohens_d_right_transp.csv', header = None)
# Remove the first row
data = data.iloc[1:]
# Remove the first two columns
data = data.iloc[:, 1:]
# Reset the index
data = data.reset_index(drop=True)
data=data.values.flatten()
ct_surf = parcel_to_surface(data, target_lab='aparc_fsa5')
d['node_yas']=ct_surf 

print(d)



In [ ]:
#Change number permutations
fsavg = nilearn.datasets.fetch_surf_fsaverage('fsaverage5')
sphere_lh_fs = surface.load_surf_mesh(fsavg['sphere_left'])[0]
sphere_rh_fs = surface.load_surf_mesh(fsavg['sphere_right'])[0]
#sp_fs = SpinPermutations(n_rep=1000, random_state=0)
sp_fs = SpinPermutations(n_rep=10, random_state=0)
sp_fs.fit(sphere_lh_fs, points_rh=sphere_rh_fs)


In [ ]:
def permute(inp):
    left = inp[:10242]
    right = inp[10242:]
    return np.hstack(sp_fs.randomize(left, right))

g1 = schaefer_to_surf(parcels, rc_g1)
g2 = schaefer_to_surf(parcels, rc_g2)
g3 = schaefer_to_surf(parcels, rc_g3)

In [ ]:
enig_g1_spin = {}
enig_g2_spin = {}
enig_g3_spin = {}
select = ['node_track', 'node_trackon', 'node_yas']

for key, val in d.items():
    if key in select:
        print(key)
        permuted = permute(val)
        enig_g1_spin[key] = spin(g1, val, permuted)
        enig_g2_spin[key] = spin(g2, val, permuted)
        enig_g3_spin[key] = spin(g3, val, permuted)

In [ ]:
def plot_corr(inp, fname):
    enigma_keys = {'Track': inp['node_track'],
                   'TrackOn': inp['node_trackon'],
                   'YAS': inp['node_yas']}
    plot = pd.DataFrame.from_dict(enigma_keys, orient='index')
    plot.columns = ['r', 'p_vgm']
    plot.sort_values(by='r', inplace=True)
    fig, ax = plt.subplots(1, figsize=[11, 12])
    color = ['lightskyblue' if x > 0.05 else 'dodgerblue' for x in plot['p_vgm']]
    ax.barh(range(len(plot)), plot['r'], color=color)
    ax.set_yticks(range(0, len(plot)))
    ax.set_yticklabels(plot.index, fontsize=32)
    ax.set_xlabel("spearman's r", fontsize=32)
    ax.tick_params(labelsize=32)
    plt.xlim([-0.50, 0.50])
    plt.tight_layout()
    fig.savefig(res_p + fname)
    return

In [ ]:
plot_corr(enig_g1_spin, 'rc_g1_mind_node.png')
plot_corr(enig_g2_spin, 'rc_g2_mind_node.png')
plot_corr(enig_g3_spin, 'rc_g3_mind_node.png')


In [ ]:
#EDGE
disorders = ['edge_track', 'edge_trackon', 'edge_yas']
d = {}


# Read the CSV file
data=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/R/mind/track/results/edge/cohens_d_right_transp.csv', header = None)
# Remove the first row
data = data.iloc[1:]
# Remove the first two columns
data = data.iloc[:, 1:]
# Reset the index
data = data.reset_index(drop=True)
data=data.values.flatten()
ct_surf = parcel_to_surface(data, target_lab='aparc_fsa5')
d['edge_track']=ct_surf 



# Read the CSV file
data=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/R/mind/trackon/results/edge/cohens_d_right_transp.csv', header = None)
# Remove the first row
data = data.iloc[1:]
# Remove the first two columns
data = data.iloc[:, 1:]
# Reset the index
data = data.reset_index(drop=True)
data=data.values.flatten()
ct_surf = parcel_to_surface(data, target_lab='aparc_fsa5')
d['edge_trackon']=ct_surf 

# Read the CSV file
data=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/R/mind/yas/results/edge/cohens_d_right_transp.csv', header = None)
# Remove the first row
data = data.iloc[1:]
# Remove the first two columns
data = data.iloc[:, 1:]
# Reset the index
data = data.reset_index(drop=True)
data=data.values.flatten()
ct_surf = parcel_to_surface(data, target_lab='aparc_fsa5')
d['edge_yas']=ct_surf 

print(d)

#Change number permutations
fsavg = nilearn.datasets.fetch_surf_fsaverage('fsaverage5')
sphere_lh_fs = surface.load_surf_mesh(fsavg['sphere_left'])[0]
sphere_rh_fs = surface.load_surf_mesh(fsavg['sphere_right'])[0]
#sp_fs = SpinPermutations(n_rep=1000, random_state=0)
sp_fs = SpinPermutations(n_rep=10, random_state=0)
sp_fs.fit(sphere_lh_fs, points_rh=sphere_rh_fs)



def permute(inp):
    left = inp[:10242]
    right = inp[10242:]
    return np.hstack(sp_fs.randomize(left, right))

g1 = schaefer_to_surf(parcels, rc_g1)
g2 = schaefer_to_surf(parcels, rc_g2)
g3 = schaefer_to_surf(parcels, rc_g3)


enig_g1_spin = {}
enig_g2_spin = {}
enig_g3_spin = {}
select = ['edge_track', 'edge_trackon', 'edge_yas']

for key, val in d.items():
    if key in select:
        print(key)
        permuted = permute(val)
        enig_g1_spin[key] = spin(g1, val, permuted)
        enig_g2_spin[key] = spin(g2, val, permuted)
        enig_g3_spin[key] = spin(g3, val, permuted)
        
def plot_corr(inp, fname):
    enigma_keys = {'Track': inp['edge_track'],
                   'TrackOn': inp['edge_trackon'],
                   'YAS': inp['edge_yas']}
    plot = pd.DataFrame.from_dict(enigma_keys, orient='index')
    plot.columns = ['r', 'p_vgm']
    plot.sort_values(by='r', inplace=True)
    fig, ax = plt.subplots(1, figsize=[11, 12])
    color = ['lightskyblue' if x > 0.05 else 'dodgerblue' for x in plot['p_vgm']]
    ax.barh(range(len(plot)), plot['r'], color=color)
    ax.set_yticks(range(0, len(plot)))
    ax.set_yticklabels(plot.index, fontsize=32)
    ax.set_xlabel("spearman's r", fontsize=32)
    ax.tick_params(labelsize=32)
    plt.xlim([-0.50, 0.50])
    plt.tight_layout()
    fig.savefig(res_p + fname)
    return      


plot_corr(enig_g1_spin, 'rc_g1_mind_edge.png')
plot_corr(enig_g2_spin, 'rc_g2_mind_edge.png')
plot_corr(enig_g3_spin, 'rc_g3_mind_edge.png')
